In [1]:
from sklearn import datasets                         # For using existing sklearn datasets 
from sklearn import tree                             # desired model
from sklearn.model_selection import train_test_split # split data to training ones and test ones 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score # model evaluation
from sklearn.metrics import accuracy_score               # model evaluation
from sklearn.model_selection import learning_curve
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from matplotlib.pyplot import figure
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image
import seaborn as sns
import time
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

%matplotlib inline

In [2]:
precip_data = pd.read_csv("excels1/OUT_precip.csv") #done
temp_data = pd.read_csv("excels1/OUT_temp.csv") #done
mix_data = pd.read_csv("excels1/OUT_inv_pma_orig_mix.csv")
section_data = pd.read_csv("excels1/OUT_mon_t_prof_index_section.csv")
histtrf_data = pd.read_csv("excels1/OUT_trf_hist_est_esal.csv") #done
montrf_data = pd.read_csv("excels1/OUT_trf_mon_est_esal.csv") #done

In [18]:
tstug_data = pd.read_csv("excels1/OUT_tst.csv")

### Precipitation and Temperature Merge

In [3]:
precip_data

,Unnamed: 0,SHRP_ID,STATE_CODE,STATE_CODE_EXP,VWS_ID,YEAR,TOTAL_ANN_PRECIP,TOTAL_SNOWFALL_YR,INTENSE_PRECIP_DAYS_YR,WET_DAYS_YR,NUM_ANN_PRECIP_DAYS,NUM_SNOWFALL_DAYS_YR,SNOW_COVERED_DAYS_YR,MAX_CONTRIBUTE_WS_NO,RECORD_STATUS,EXPANDED_STATUS
0,0,0607,1,Alabama,010600,1950,1513.099976,23,40,144,365,365,0,3,E,1
1,1,0608,1,Alabama,010600,1950,1513.099976,23,40,144,365,365,0,3,E,1
2,2,0661,1,Alabama,010600,1950,1513.099976,23,40,144,365,365,0,3,E,1
3,3,0662,1,Alabama,010600,1950,1513.099976,23,40,144,365,365,0,3,E,1
4,4,0663,1,Alabama,010600,1950,1513.099976,23,40,144,365,365,0,3,E,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123407,123407,0901,90,Saskatchewan,900900,1977,386.500000,680,7,79,365,365,0,5,E,1
123408,123408,0902,90,Saskatchewan,900900,1977,386.500000,680,7,79,365,365,0,5,E,1
123409,123409,0903,90,Saskatchewan,900900,1977,386.500000,680,7,79,365,365,0,5,E,1
123410,123410,0959,90,Saskatchewan,900900,1977,386.500000,680,7,79,365,365,0,5,E,1


In [4]:
precip_data['SHRP_ID'].unique().size

1028

In [5]:
precip_data.columns

Index(['Unnamed: 0', 'SHRP_ID', 'STATE_CODE', 'STATE_CODE_EXP', 'VWS_ID',
       'YEAR', 'TOTAL_ANN_PRECIP', 'TOTAL_SNOWFALL_YR',
       'INTENSE_PRECIP_DAYS_YR', 'WET_DAYS_YR', 'NUM_ANN_PRECIP_DAYS',
       'NUM_SNOWFALL_DAYS_YR', 'SNOW_COVERED_DAYS_YR', 'MAX_CONTRIBUTE_WS_NO',
       'RECORD_STATUS', 'EXPANDED_STATUS'],
      dtype='object')

In [6]:
precip_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [7]:
preciptemp_merge = precip_data.merge(temp_data, on=['SHRP_ID', 'STATE_CODE', 'YEAR'], how='outer', suffixes=['_p', '_t'])
preciptemp_merge

,SHRP_ID,STATE_CODE,STATE_CODE_EXP_p,VWS_ID_p,YEAR,TOTAL_ANN_PRECIP,TOTAL_SNOWFALL_YR,INTENSE_PRECIP_DAYS_YR,WET_DAYS_YR,NUM_ANN_PRECIP_DAYS,...,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t
0,0607,1,Alabama,010600,1950,1513.099976,23.0,40.0,144.0,365.0,...,365.0,33.400002,-18.100000,7.0,55.0,365.0,365.0,3.0,E,1.0
1,0608,1,Alabama,010600,1950,1513.099976,23.0,40.0,144.0,365.0,...,365.0,33.400002,-18.100000,7.0,55.0,365.0,365.0,3.0,E,1.0
2,0661,1,Alabama,010600,1950,1513.099976,23.0,40.0,144.0,365.0,...,365.0,33.400002,-18.100000,7.0,55.0,365.0,365.0,3.0,E,1.0
3,0662,1,Alabama,010600,1950,1513.099976,23.0,40.0,144.0,365.0,...,365.0,33.400002,-18.100000,7.0,55.0,365.0,365.0,3.0,E,1.0
4,0663,1,Alabama,010600,1950,1513.099976,23.0,40.0,144.0,365.0,...,365.0,33.400002,-18.100000,7.0,55.0,365.0,365.0,3.0,E,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123408,0902,90,Saskatchewan,900900,1977,386.500000,680.0,7.0,79.0,365.0,...,365.0,33.200001,-41.000000,1.0,190.0,365.0,365.0,5.0,E,1.0
123409,0903,90,Saskatchewan,900900,1977,386.500000,680.0,7.0,79.0,365.0,...,365.0,33.200001,-41.000000,1.0,190.0,365.0,365.0,5.0,E,1.0
123410,0959,90,Saskatchewan,900900,1977,386.500000,680.0,7.0,79.0,365.0,...,365.0,33.200001,-41.000000,1.0,190.0,365.0,365.0,5.0,E,1.0
123411,0960,90,Saskatchewan,900900,1977,386.500000,680.0,7.0,79.0,365.0,...,365.0,33.200001,-41.000000,1.0,190.0,365.0,365.0,5.0,E,1.0


### Traffic data merge (Mon and Hist)

In [8]:
histtrf_data.rename(columns={'YEAR_HIST_EST': 'YEAR'}, inplace=True)
histtrf_data

,Unnamed: 0,STATE_CODE,STATE_CODE_EXP,SHRP_ID,YEAR,RECORD_STATUS,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR
0,0,42,Pennsylvania,7037,1986,E,13696,5294,5478,2118,710
1,1,42,Pennsylvania,7037,1987,E,14106,5452,5642,2180,730
2,2,42,Pennsylvania,7037,1988,E,14530,5616,5812,2246,752
3,3,42,Pennsylvania,7037,1989,E,15257,5896,6102,2358,790
4,4,44,Rhode Island,7401,1986,E,9400,1320,3480,480,175
...,...,...,...,...,...,...,...,...,...,...,...
20417,20417,4,Arizona,B960,1979,D,7800,2340,3120,936,1103
20418,20418,4,Arizona,B961,1979,D,7800,2340,3120,936,1103
20419,20419,4,Arizona,B964,1979,D,7800,2340,3120,936,1103
20420,20420,4,Arizona,1007,1980,D,7700,2310,3080,924,1089


In [9]:
histtrf_data.drop(['Unnamed: 0'], inplace=True, axis=1)

In [10]:
years = histtrf_data['YEAR'].unique()
years.sort()
years

array([1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975,
       1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989])

In [13]:
years1 = montrf_data['YEAR'].unique()
years1.sort()
years1

array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [11]:
montrf_data.rename(columns={'YEAR_MON_EST': 'YEAR'}, inplace=True)
montrf_data

,Unnamed: 0,STATE_CODE,STATE_CODE_EXP,SHRP_ID,YEAR,RECORD_STATUS,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR
0,0,51,Virginia,2021,1991,E,8800,671,3520,268,75
1,1,51,Virginia,5008,1990,E,52390,1676,6549,85,31
2,2,51,Virginia,5009,1990,E,10210,830,4084,332,93
3,3,54,West Virginia,1640,1992,E,10750,957,4615,415,121
4,4,54,West Virginia,4003,1990,E,9520,847,4047,364,106
...,...,...,...,...,...,...,...,...,...,...,...
28317,28317,28,Mississippi,3085,2001,E,0,0,0,31,25
28318,28318,28,Mississippi,3085,2002,E,0,0,0,32,25
28319,28319,28,Mississippi,3085,2003,E,0,0,0,33,26
28320,28320,28,Mississippi,3085,2004,E,0,0,0,34,27


In [12]:
montrf_data.drop(['Unnamed: 0'], inplace=True, axis=1)

In [13]:
#monhist_merged1 = histtrf_data.merge(montrf_data, left_on=['STATE_CODE', 'SHRP_ID', 'YEAR_HIST_EST'], right_on=['STATE_CODE', 'SHRP_ID', 'YEAR_MON_EST'], how='outer')
monhist_merged1 = histtrf_data.merge(montrf_data, on=['STATE_CODE', 'SHRP_ID', 'YEAR'], how='outer')
monhist_merged1

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY_x,AADT_TRUCK_COMBO_2WAY_x,AADT_ALL_VEHIC_x,AADT_TRUCK_COMBO_x,ANL_KESAL_LTPP_LN_YR_x,STATE_CODE_EXP_y,RECORD_STATUS_y,AADT_ALL_VEHIC_2WAY_y,AADT_TRUCK_COMBO_2WAY_y,AADT_ALL_VEHIC_y,AADT_TRUCK_COMBO_y,ANL_KESAL_LTPP_LN_YR_y
0,42,Pennsylvania,7037,1986,E,13696.0,5294.0,5478.0,2118.0,710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42,Pennsylvania,7037,1987,E,14106.0,5452.0,5642.0,2180.0,730.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42,Pennsylvania,7037,1988,E,14530.0,5616.0,5812.0,2246.0,752.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42,Pennsylvania,7037,1989,E,15257.0,5896.0,6102.0,2358.0,790.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44,Rhode Island,7401,1986,E,9400.0,1320.0,3480.0,480.0,175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,28,NaN,3085,2001,NaN,NaN,NaN,NaN,NaN,NaN,Mississippi,E,0.0,0.0,0.0,31.0,25.0
48740,28,NaN,3085,2002,NaN,NaN,NaN,NaN,NaN,NaN,Mississippi,E,0.0,0.0,0.0,32.0,25.0
48741,28,NaN,3085,2003,NaN,NaN,NaN,NaN,NaN,NaN,Mississippi,E,0.0,0.0,0.0,33.0,26.0
48742,28,NaN,3085,2004,NaN,NaN,NaN,NaN,NaN,NaN,Mississippi,E,0.0,0.0,0.0,34.0,27.0


In [14]:
monhist_merged = pd.concat([histtrf_data, montrf_data])
monhist_merged

,STATE_CODE,STATE_CODE_EXP,SHRP_ID,YEAR,RECORD_STATUS,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR
0,42,Pennsylvania,7037,1986,E,13696,5294,5478,2118,710
1,42,Pennsylvania,7037,1987,E,14106,5452,5642,2180,730
2,42,Pennsylvania,7037,1988,E,14530,5616,5812,2246,752
3,42,Pennsylvania,7037,1989,E,15257,5896,6102,2358,790
4,44,Rhode Island,7401,1986,E,9400,1320,3480,480,175
...,...,...,...,...,...,...,...,...,...,...
28317,28,Mississippi,3085,2001,E,0,0,0,31,25
28318,28,Mississippi,3085,2002,E,0,0,0,32,25
28319,28,Mississippi,3085,2003,E,0,0,0,33,26
28320,28,Mississippi,3085,2004,E,0,0,0,34,27


### Mix data merge with ???

In [15]:
mix_data

,Unnamed: 0,SHRP_ID,STATE_CODE,CONSTRUCTION_NO,LAYER_NO,SAMPLE_TYPE,ASPHALT_CONTENT_MEAN,PCT_AIR_VOIDS_MEAN,VOIDS_MINERAL_AGGR,MARSHALL_STABILITY,MARSHALL_FLOW
0,0,5283,48,1,3,1,6.0,2.7,18.5,0,0
1,1,5274,48,1,3,1,6.0,0.0,0.0,0,0
2,2,5317,48,1,3,1,6.0,15.0,0.0,0,0
3,3,5287,48,1,3,1,5.4,0.0,0.0,0,0
4,4,5301,48,1,3,1,5.5,10.6,23.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
986,986,6160,48,1,4,1,6.1,0.0,0.0,0,0
987,987,6160,48,1,6,1,5.1,0.0,0.0,0,0
988,988,4142,48,1,3,1,7.0,0.0,0.0,0,0
989,989,1183,48,1,3,1,6.9,10.8,0.0,0,0


In [16]:
section_data

,VISIT_NO,RUN_NUMBER,STATE_CODE,STATE_CODE_EXP,SHRP_ID,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS,MAX_MEAN_DEPTH_1_8
0,24A33012,1,24,Maryland,A330,04/07/1998,2,E,5
1,24240102,1,24,Maryland,2401,03/21/1991,1,E,4
2,26012105,1,26,Michigan,0121,10/13/2009,4,E,5
3,24056206,1,24,Maryland,0562,06/21/1999,2,E,4
4,24050706,1,24,Maryland,0507,10/17/1995,2,E,5
...,...,...,...,...,...,...,...,...,...
21631,24090107,1,24,Maryland,0901,01/03/2004,2,E,4
21632,27090310,1,27,Minnesota,0903,09/09/2005,4,E,8
21633,27102303,1,27,Minnesota,1023,05/30/1990,1,E,8
21634,27625109,1,27,Minnesota,6251,06/13/1995,1,E,7


In [19]:
tstug_data

,Unnamed: 0,SHRP_ID,STATE_CODE,STATE_CODE_EXP,LAYER_NO,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,...,DEF_LVDT_2_AVG,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_AVG,RES_MOD_STD,TEST_DATE,RECORD_STATUS
0,0,1647,88,Prince Edward Island,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,...,0.11367,NaN,0.11379,0.00008,0.000795,NaN,78,0.0,1992-11-12 00:00:00,E
1,1,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,...,0.02700,NaN,0.02659,0.00008,0.000183,NaN,66,1.0,1992-11-25 00:00:00,E
2,2,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,...,0.08367,NaN,0.08557,0.00013,0.000590,NaN,86,0.0,1992-11-25 00:00:00,E
3,3,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,...,0.07605,NaN,0.07810,0.00010,0.000538,NaN,71,0.0,1992-11-25 00:00:00,E
4,4,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,...,0.06177,NaN,0.06487,0.00008,0.000447,NaN,56,0.0,1992-11-25 00:00:00,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155362,155362,1053,8,Colorado,1,2,Material sample obtained from area adjacent to...,1,A2,JS03,...,0.17574,0.00020,0.17694,0.00010,0.001167,0.000000e+00,43,0.0,1995-09-20 00:00:00,E
155363,155363,1053,8,Colorado,1,2,Material sample obtained from area adjacent to...,1,A2,JS03,...,0.22079,0.00011,0.22262,0.00018,0.001468,1.000000e-06,42,0.0,1995-09-20 00:00:00,E
155364,155364,7079,4,Arizona,1,1,Material sample obtained from area adjacent to...,1,TP1,BS92,...,0.02932,0.00012,0.02722,0.00009,0.000178,0.000000e+00,70,0.0,1996-06-18 00:00:00,E
155365,155365,7079,4,Arizona,1,1,Material sample obtained from area adjacent to...,1,TP1,BS92,...,0.07288,0.00014,0.07039,0.00009,0.000460,1.000000e-06,81,0.0,1996-06-18 00:00:00,E


In [20]:
tstug_data['YEAR'] = pd.DatetimeIndex(tstug_data['TEST_DATE']).year
tstug_data.drop(['Unnamed: 0'], inplace=True, axis=1)
tstug_data

,SHRP_ID,STATE_CODE,STATE_CODE_EXP,LAYER_NO,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,CON_PRESSURE,...,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_AVG,RES_MOD_STD,TEST_DATE,RECORD_STATUS,YEAR
0,1647,88,Prince Edward Island,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,41.400002,...,NaN,0.11379,0.00008,0.000795,NaN,78,0.0,1992-11-12 00:00:00,E,1992
1,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,41.400002,...,NaN,0.02659,0.00008,0.000183,NaN,66,1.0,1992-11-25 00:00:00,E,1992
2,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,41.400002,...,NaN,0.08557,0.00013,0.000590,NaN,86,0.0,1992-11-25 00:00:00,E,1992
3,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,27.600000,...,NaN,0.07810,0.00010,0.000538,NaN,71,0.0,1992-11-25 00:00:00,E,1992
4,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,13.800000,...,NaN,0.06487,0.00008,0.000447,NaN,56,0.0,1992-11-25 00:00:00,E,1992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155362,1053,8,Colorado,1,2,Material sample obtained from area adjacent to...,1,A2,JS03,13.800000,...,0.00020,0.17694,0.00010,0.001167,0.000000e+00,43,0.0,1995-09-20 00:00:00,E,1995
155363,1053,8,Colorado,1,2,Material sample obtained from area adjacent to...,1,A2,JS03,13.800000,...,0.00011,0.22262,0.00018,0.001468,1.000000e-06,42,0.0,1995-09-20 00:00:00,E,1995
155364,7079,4,Arizona,1,1,Material sample obtained from area adjacent to...,1,TP1,BS92,41.299999,...,0.00012,0.02722,0.00009,0.000178,0.000000e+00,70,0.0,1996-06-18 00:00:00,E,1996
155365,7079,4,Arizona,1,1,Material sample obtained from area adjacent to...,1,TP1,BS92,41.299999,...,0.00014,0.07039,0.00009,0.000460,1.000000e-06,81,0.0,1996-06-18 00:00:00,E,1996


### Section data merge with new tables

In [21]:
section_data['YEAR'] = pd.DatetimeIndex(section_data['SURVEY_DATE']).year
section_data

,VISIT_NO,RUN_NUMBER,STATE_CODE,STATE_CODE_EXP,SHRP_ID,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS,MAX_MEAN_DEPTH_1_8,YEAR
0,24A33012,1,24,Maryland,A330,04/07/1998,2,E,5,1998
1,24240102,1,24,Maryland,2401,03/21/1991,1,E,4,1991
2,26012105,1,26,Michigan,0121,10/13/2009,4,E,5,2009
3,24056206,1,24,Maryland,0562,06/21/1999,2,E,4,1999
4,24050706,1,24,Maryland,0507,10/17/1995,2,E,5,1995
...,...,...,...,...,...,...,...,...,...,...
21631,24090107,1,24,Maryland,0901,01/03/2004,2,E,4,2004
21632,27090310,1,27,Minnesota,0903,09/09/2005,4,E,8,2005
21633,27102303,1,27,Minnesota,1023,05/30/1990,1,E,8,1990
21634,27625109,1,27,Minnesota,6251,06/13/1995,1,E,7,1995


In [22]:
years2 = section_data['YEAR'].unique()
years2.sort()
years2

array([1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
       2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [23]:
trfsection_merged = monhist_merged.merge(section_data, on=['STATE_CODE', 'SHRP_ID', 'YEAR'], how='inner')
trfsection_merged

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,VISIT_NO,RUN_NUMBER,STATE_CODE_EXP_y,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS_y,MAX_MEAN_DEPTH_1_8
0,42,Pennsylvania,7037,1989,E,15257,5896,6102,2358,790,42703701,1,Pennsylvania,08/29/1989,1,E,4
1,44,Rhode Island,7401,1989,E,10000,1400,3700,510,186,44740101,1,Rhode Island,08/04/1989,1,E,3
2,42,Pennsylvania,9027,1989,E,15913,5070,6365,2028,1012,42902701,1,Pennsylvania,07/24/1989,1,E,3
3,23,Maine,7023,1989,E,33540,3720,10900,1860,774,23702301,1,Maine,08/10/1989,1,E,5
4,51,Virginia,2564,1989,E,39585,5235,15834,2094,589,51256401,1,Virginia,10/12/1989,1,E,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12107,28,Mississippi,3083,2003,E,0,0,0,33,25,28308312,1,Mississippi,02/25/2003,1,E,2
12108,28,Mississippi,3085,1999,E,0,0,0,29,23,28308509,1,Mississippi,09/14/1999,2,E,3
12109,28,Mississippi,3085,2001,E,0,0,0,31,25,28308510,1,Mississippi,03/05/2001,2,E,4
12110,28,Mississippi,3085,2003,E,0,0,0,33,26,28308511,1,Mississippi,02/25/2003,3,E,3


In [24]:
trfsection_merged[trfsection_merged['SHRP_ID'] == '7037']

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,VISIT_NO,RUN_NUMBER,STATE_CODE_EXP_y,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS_y,MAX_MEAN_DEPTH_1_8
0,42,Pennsylvania,7037,1989,E,15257,5896,6102,2358,790,42703701,1,Pennsylvania,08/29/1989,1,E,4
988,42,Pennsylvania,7037,1991,E,16773,5217,5870,1826,678,42703703,1,Pennsylvania,08/07/1991,2,E,3
989,42,Pennsylvania,7037,1993,E,19025,6483,6659,2269,945,42703705,1,Pennsylvania,03/02/1993,2,E,5
1187,42,Pennsylvania,7037,1990,E,16373,5092,5730,1782,661,42703702,1,Pennsylvania,09/27/1990,2,E,3
1602,42,Pennsylvania,7037,2000,E,24250,9606,8487,3362,1230,42703708,1,Pennsylvania,07/26/2000,3,E,5
1603,42,Pennsylvania,7037,2000,E,24250,9606,8487,3362,1230,42703709,1,Pennsylvania,08/24/2000,3,E,5
1726,42,Pennsylvania,7037,1998,E,23427,9283,8199,3249,1189,42703707,1,Pennsylvania,03/24/1998,3,E,3


### merge temp precip with section trf

In [25]:
temptrfsection_merged = trfsection_merged.merge(preciptemp_merge, on=['STATE_CODE', 'SHRP_ID', 'YEAR'], how='inner')
temptrfsection_merged

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,...,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t
0,42,Pennsylvania,7037,1989,E,15257,5896,6102,2358,790,...,365.0,31.700001,-27.299999,0.0,165.0,365.0,365.0,6.0,E,0.0
1,44,Rhode Island,7401,1989,E,10000,1400,3700,510,186,...,365.0,33.599998,-22.000000,3.0,145.0,365.0,365.0,6.0,E,0.0
2,42,Pennsylvania,9027,1989,E,15913,5070,6365,2028,1012,...,365.0,33.400002,-17.500000,8.0,134.0,365.0,365.0,7.0,E,0.0
3,23,Maine,7023,1989,E,33540,3720,10900,1860,774,...,365.0,35.400002,-25.600000,4.0,152.0,365.0,365.0,5.0,E,0.0
4,51,Virginia,2564,1989,E,39585,5235,15834,2094,589,...,365.0,36.099998,-13.400000,26.0,62.0,365.0,365.0,5.0,E,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12047,28,Mississippi,3083,2003,E,0,0,0,33,25,...,365.0,35.599998,-14.400000,40.0,72.0,365.0,365.0,7.0,E,0.0
12048,28,Mississippi,3085,1999,E,0,0,0,29,23,...,365.0,38.099998,-14.300000,70.0,57.0,365.0,365.0,5.0,E,0.0
12049,28,Mississippi,3085,2001,E,0,0,0,31,25,...,365.0,35.599998,-14.900000,42.0,70.0,365.0,365.0,5.0,E,0.0
12050,28,Mississippi,3085,2003,E,0,0,0,33,26,...,365.0,35.099998,-14.800000,34.0,74.0,365.0,365.0,5.0,E,0.0


In [26]:
temptrfsection_merged[temptrfsection_merged['SHRP_ID'] == '1640']#).shape[0]

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,...,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t
158,54,West Virginia,1640,1989,E,10900,800,4524,362,82,...,365.0,33.700001,-24.900000,10.0,106.0,365.0,365.0,5.0,E,0.0
608,54,West Virginia,1640,1992,E,10750,957,4615,415,121,...,366.0,33.500000,-14.800000,6.0,97.0,366.0,366.0,5.0,E,0.0
721,54,West Virginia,1640,2001,E,16377,1458,6763,609,168,...,365.0,34.200001,-14.300000,9.0,118.0,365.0,365.0,6.0,E,0.0
1361,54,West Virginia,1640,1991,E,10140,902,4331,390,114,...,365.0,35.799999,-14.500000,30.0,95.0,365.0,365.0,5.0,E,0.0
1604,54,West Virginia,1640,2000,E,14354,1278,5754,518,151,...,366.0,33.900002,-19.799999,3.0,118.0,366.0,366.0,6.0,E,0.0
1629,54,West Virginia,1640,1998,E,12249,1090,4908,444,123,...,365.0,35.799999,-13.100000,11.0,88.0,365.0,365.0,6.0,E,0.0
2561,54,West Virginia,1640,2004,E,16873,1502,6968,627,173,...,366.0,32.299999,-18.600000,1.0,104.0,366.0,366.0,6.0,E,0.0


In [27]:
'LAYER_NO_x' in temptrfsection_merged.columns

False

In [28]:
tstug_data

,SHRP_ID,STATE_CODE,STATE_CODE_EXP,LAYER_NO,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,CON_PRESSURE,...,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_AVG,RES_MOD_STD,TEST_DATE,RECORD_STATUS,YEAR
0,1647,88,Prince Edward Island,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,41.400002,...,NaN,0.11379,0.00008,0.000795,NaN,78,0.0,1992-11-12 00:00:00,E,1992
1,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,41.400002,...,NaN,0.02659,0.00008,0.000183,NaN,66,1.0,1992-11-25 00:00:00,E,1992
2,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,41.400002,...,NaN,0.08557,0.00013,0.000590,NaN,86,0.0,1992-11-25 00:00:00,E,1992
3,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,27.600000,...,NaN,0.07810,0.00010,0.000538,NaN,71,0.0,1992-11-25 00:00:00,E,1992
4,4008,9,Connecticut,1,1,Material sample obtained from area adjacent to...,1,BA3,BS03,13.800000,...,NaN,0.06487,0.00008,0.000447,NaN,56,0.0,1992-11-25 00:00:00,E,1992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155362,1053,8,Colorado,1,2,Material sample obtained from area adjacent to...,1,A2,JS03,13.800000,...,0.00020,0.17694,0.00010,0.001167,0.000000e+00,43,0.0,1995-09-20 00:00:00,E,1995
155363,1053,8,Colorado,1,2,Material sample obtained from area adjacent to...,1,A2,JS03,13.800000,...,0.00011,0.22262,0.00018,0.001468,1.000000e-06,42,0.0,1995-09-20 00:00:00,E,1995
155364,7079,4,Arizona,1,1,Material sample obtained from area adjacent to...,1,TP1,BS92,41.299999,...,0.00012,0.02722,0.00009,0.000178,0.000000e+00,70,0.0,1996-06-18 00:00:00,E,1996
155365,7079,4,Arizona,1,1,Material sample obtained from area adjacent to...,1,TP1,BS92,41.299999,...,0.00014,0.07039,0.00009,0.000460,1.000000e-06,81,0.0,1996-06-18 00:00:00,E,1996


In [29]:
tstug_data[(tstug_data['SHRP_ID'] == '1640') & (tstug_data['LAYER_TYPE'] == 1)]#).shape[0]

,SHRP_ID,STATE_CODE,STATE_CODE_EXP,LAYER_NO,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,CON_PRESSURE,...,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_AVG,RES_MOD_STD,TEST_DATE,RECORD_STATUS,YEAR
163,1640,54,West Virginia,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,13.800000,...,NaN,0.20422,0.00008,0.001410,NaN,22,0.0,1992-12-04 00:00:00,E,1992
1782,1640,54,West Virginia,1,1,Material sample obtained from area adjacent to...,1,BA*,BS**,41.400002,...,NaN,0.07460,0.00010,0.000517,NaN,121,0.0,1992-12-03 00:00:00,D,1992
1783,1640,54,West Virginia,1,1,Material sample obtained from area adjacent to...,1,BA*,BS**,13.800000,...,NaN,0.01605,0.00008,0.000111,NaN,113,1.0,1992-12-03 00:00:00,E,1992
18073,1640,54,West Virginia,1,1,Material sample obtained from area adjacent to...,1,BA*,BS**,27.600000,...,NaN,0.08240,0.00005,0.000571,NaN,109,0.0,1992-12-03 00:00:00,E,1992
22337,1640,54,West Virginia,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,41.400002,...,NaN,0.17828,0.00005,0.001231,NaN,26,0.0,1992-12-04 00:00:00,E,1992
22338,1640,54,West Virginia,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,27.600000,...,NaN,0.33002,0.00020,0.002278,NaN,24,0.0,1992-12-04 00:00:00,E,1992
39013,1640,54,West Virginia,1,1,Material sample obtained from area adjacent to...,1,BA*,BS**,13.800000,...,NaN,0.03348,0.00015,0.000323,NaN,107,0.0,1992-12-03 00:00:00,D,1992
39014,1640,54,West Virginia,1,1,Material sample obtained from area adjacent to...,1,BA*,BS**,13.800000,...,NaN,0.07430,0.00008,0.000515,NaN,96,0.0,1992-12-03 00:00:00,E,1992
39015,1640,54,West Virginia,1,1,Material sample obtained from area adjacent to...,1,BA*,BS**,13.800000,...,NaN,0.09390,0.00010,0.000651,NaN,95,0.0,1992-12-03 00:00:00,E,1992
39688,1640,54,West Virginia,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,27.600000,...,NaN,0.03599,0.00000,0.000248,NaN,48,0.0,1992-12-04 00:00:00,E,1992


In [30]:
tstug_data = tstug_data[tstug_data['LAYER_TYPE'] == 1]

### Merged with tst

In [31]:
pd.set_option('display.max_columns', None)

In [56]:
allbutmix_merged = temptrfsection_merged.merge(tstug_data, on=['STATE_CODE', 'SHRP_ID', 'YEAR', 'CONSTRUCTION_NO'], how='inner')
allbutmix_merged

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,VISIT_NO,RUN_NUMBER,STATE_CODE_EXP_y,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS_y,MAX_MEAN_DEPTH_1_8,STATE_CODE_EXP_p,VWS_ID_p,TOTAL_ANN_PRECIP,TOTAL_SNOWFALL_YR,INTENSE_PRECIP_DAYS_YR,WET_DAYS_YR,NUM_ANN_PRECIP_DAYS,NUM_SNOWFALL_DAYS_YR,SNOW_COVERED_DAYS_YR,MAX_CONTRIBUTE_WS_NO_p,RECORD_STATUS_p,EXPANDED_STATUS_p,Unnamed: 0,STATE_CODE_EXP_t,VWS_ID_t,MEAN_ANN_TEMP_AVG,FREEZE_INDEX_YR,FREEZE_THAW_YR,MEAN_ANN_TEMP_DAYS,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,MIN_ANN_TEMP_AVG,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t,STATE_CODE_EXP,LAYER_NO,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,CON_PRESSURE,NOM_MAX_AXIAL_STRESS,LAB_CODE,LAB_CODE_EXP,SAMPLE_AREA_NO,LAYER_TYPE,LAYER_TYPE_EXP,MR_MATL_TYPE,MR_MATL_TYPE_EXP,APPLIED_MAX_AXIAL_LOAD_AVG,APPLIED_MAX_AXIAL_LOAD_STD,APPLIED_CYCLIC_LOAD_AVG,APPLIED_CYCLIC_LOAD_STD,APPLIED_CONTACT_LOAD_AVG,APPLIED_CONTACT_LOAD_STD,APPLIED_MAX_AXIAL_STRESS_AVG,APPLIED_MAX_AXIAL_STRESS_STD,APPLIED_CYCLIC_STRESS_AVG,APPLIED_CYCLIC_STRESS_STD,APPLIED_CONTACT_STRESS_AVG,APPLIED_CONTACT_STRESS_STD,DEF_LVDT_1_AVG,DEF_LVDT_1_STD,DEF_LVDT_2_AVG,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_AVG,RES_MOD_STD,TEST_DATE,RECORD_STATUS
0,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,41.400002,41.400002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,146.899994,0.4,NaN,NaN,NaN,NaN,36.900002,NaN,4.1,NaN,0.25212,NaN,0.26464,NaN,0.25839,0.00013,0.001374,NaN,21,0.0,1992-11-18 00:00:00,D
1,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,27.600000,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,230.199997,0.7,NaN,NaN,NaN,NaN,57.799999,NaN,6.9,NaN,0.45397,NaN,0.47648,NaN,0.46523,0.00036,0.003122,NaN,19,0.0,1992-11-18 00:00:00,E
2,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,1,Material sample obtained from area adjacent to...,1,BA2,BS02,41.400002,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,245.899994,0.3,NaN,NaN,NaN,NaN,62.000000,NaN,6.9,NaN,0.10559,NaN,0.12616,NaN,0.11587,0.00015,0.000802,NaN,77,0.0,1992-11-18 00:00:00,E
3,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,1,Material sample obtained from area adjacent to...,1,BA2,BS02,27.600000,55.200001,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,197.800003,0.2,NaN,NaN,NaN,NaN,49.900002,NaN,5.5,NaN,0.08222,NaN,0.10056,NaN,0.09139,0.00008,0.000633,NaN,79,0.0,1992-11-18 00:00:00,E
4,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102

In [49]:
allbutmix_merged[allbutmix_merged['SHRP_ID'] == '4008']

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,VISIT_NO,RUN_NUMBER,STATE_CODE_EXP_y,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS_y,MAX_MEAN_DEPTH_1_8,STATE_CODE_EXP_p,VWS_ID_p,TOTAL_ANN_PRECIP,TOTAL_SNOWFALL_YR,INTENSE_PRECIP_DAYS_YR,WET_DAYS_YR,NUM_ANN_PRECIP_DAYS,NUM_SNOWFALL_DAYS_YR,SNOW_COVERED_DAYS_YR,MAX_CONTRIBUTE_WS_NO_p,RECORD_STATUS_p,EXPANDED_STATUS_p,Unnamed: 0,STATE_CODE_EXP_t,VWS_ID_t,MEAN_ANN_TEMP_AVG,FREEZE_INDEX_YR,FREEZE_THAW_YR,MEAN_ANN_TEMP_DAYS,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,MIN_ANN_TEMP_AVG,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t,STATE_CODE_EXP,LAYER_NO,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,CON_PRESSURE,NOM_MAX_AXIAL_STRESS,LAB_CODE,LAB_CODE_EXP,SAMPLE_AREA_NO,LAYER_TYPE,LAYER_TYPE_EXP,MR_MATL_TYPE,MR_MATL_TYPE_EXP,APPLIED_MAX_AXIAL_LOAD_AVG,APPLIED_MAX_AXIAL_LOAD_STD,APPLIED_CYCLIC_LOAD_AVG,APPLIED_CYCLIC_LOAD_STD,APPLIED_CONTACT_LOAD_AVG,APPLIED_CONTACT_LOAD_STD,APPLIED_MAX_AXIAL_STRESS_AVG,APPLIED_MAX_AXIAL_STRESS_STD,APPLIED_CYCLIC_STRESS_AVG,APPLIED_CYCLIC_STRESS_STD,APPLIED_CONTACT_STRESS_AVG,APPLIED_CONTACT_STRESS_STD,DEF_LVDT_1_AVG,DEF_LVDT_1_STD,DEF_LVDT_2_AVG,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_AVG,RES_MOD_STD,TEST_DATE,RECORD_STATUS


In [34]:
allbutmix_merged[allbutmix_merged['SHRP_ID'] == '1024']

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,VISIT_NO,RUN_NUMBER,STATE_CODE_EXP_y,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS_y,MAX_MEAN_DEPTH_1_8,STATE_CODE_EXP_p,VWS_ID_p,TOTAL_ANN_PRECIP,TOTAL_SNOWFALL_YR,INTENSE_PRECIP_DAYS_YR,WET_DAYS_YR,NUM_ANN_PRECIP_DAYS,NUM_SNOWFALL_DAYS_YR,SNOW_COVERED_DAYS_YR,MAX_CONTRIBUTE_WS_NO_p,RECORD_STATUS_p,EXPANDED_STATUS_p,Unnamed: 0,STATE_CODE_EXP_t,VWS_ID_t,MEAN_ANN_TEMP_AVG,FREEZE_INDEX_YR,FREEZE_THAW_YR,MEAN_ANN_TEMP_DAYS,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,MIN_ANN_TEMP_AVG,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t,STATE_CODE_EXP,LAYER_NO,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,CON_PRESSURE,NOM_MAX_AXIAL_STRESS,LAB_CODE,LAB_CODE_EXP,SAMPLE_AREA_NO,LAYER_TYPE,LAYER_TYPE_EXP,MR_MATL_TYPE,MR_MATL_TYPE_EXP,APPLIED_MAX_AXIAL_LOAD_AVG,APPLIED_MAX_AXIAL_LOAD_STD,APPLIED_CYCLIC_LOAD_AVG,APPLIED_CYCLIC_LOAD_STD,APPLIED_CONTACT_LOAD_AVG,APPLIED_CONTACT_LOAD_STD,APPLIED_MAX_AXIAL_STRESS_AVG,APPLIED_MAX_AXIAL_STRESS_STD,APPLIED_CYCLIC_STRESS_AVG,APPLIED_CYCLIC_STRESS_STD,APPLIED_CONTACT_STRESS_AVG,APPLIED_CONTACT_STRESS_STD,DEF_LVDT_1_AVG,DEF_LVDT_1_STD,DEF_LVDT_2_AVG,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_AVG,RES_MOD_STD,TEST_DATE,RECORD_STATUS
0,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,41.400002,41.400002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,146.899994,0.4,NaN,NaN,NaN,NaN,36.900002,NaN,4.1,NaN,0.25212,NaN,0.26464,NaN,0.25839,0.00013,0.001374,NaN,21,0.0,1992-11-18 00:00:00,D
1,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,27.600000,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,230.199997,0.7,NaN,NaN,NaN,NaN,57.799999,NaN,6.9,NaN,0.45397,NaN,0.47648,NaN,0.46523,0.00036,0.003122,NaN,19,0.0,1992-11-18 00:00:00,E
2,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,1,Material sample obtained from area adjacent to...,1,BA2,BS02,41.400002,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,245.899994,0.3,NaN,NaN,NaN,NaN,62.000000,NaN,6.9,NaN,0.10559,NaN,0.12616,NaN,0.11587,0.00015,0.000802,NaN,77,0.0,1992-11-18 00:00:00,E
3,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,1,Material sample obtained from area adjacent to...,1,BA2,BS02,27.600000,55.200001,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,197.800003,0.2,NaN,NaN,NaN,NaN,49.900002,NaN,5.5,NaN,0.08222,NaN,0.10056,NaN,0.09139,0.00008,0.000633,NaN,79,0.0,1992-11-18 00:00:00,E
4,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102

In [35]:
allbutmix_merged['MR_MATL_TYPE_EXP'].unique()

array(['Fine grained material', 'Coarse grained material'], dtype=object)

In [36]:
allbutmix_merged.columns

Index(['STATE_CODE', 'STATE_CODE_EXP_x', 'SHRP_ID', 'YEAR', 'RECORD_STATUS_x',
       'AADT_ALL_VEHIC_2WAY', 'AADT_TRUCK_COMBO_2WAY', 'AADT_ALL_VEHIC',
       'AADT_TRUCK_COMBO', 'ANL_KESAL_LTPP_LN_YR', 'VISIT_NO', 'RUN_NUMBER',
       'STATE_CODE_EXP_y', 'SURVEY_DATE', 'CONSTRUCTION_NO', 'RECORD_STATUS_y',
       'MAX_MEAN_DEPTH_1_8', 'STATE_CODE_EXP_p', 'VWS_ID_p',
       'TOTAL_ANN_PRECIP', 'TOTAL_SNOWFALL_YR', 'INTENSE_PRECIP_DAYS_YR',
       'WET_DAYS_YR', 'NUM_ANN_PRECIP_DAYS', 'NUM_SNOWFALL_DAYS_YR',
       'SNOW_COVERED_DAYS_YR', 'MAX_CONTRIBUTE_WS_NO_p', 'RECORD_STATUS_p',
       'EXPANDED_STATUS_p', 'Unnamed: 0', 'STATE_CODE_EXP_t', 'VWS_ID_t',
       'MEAN_ANN_TEMP_AVG', 'FREEZE_INDEX_YR', 'FREEZE_THAW_YR',
       'MEAN_ANN_TEMP_DAYS', 'MAX_ANN_TEMP_AVG', 'MAX_ANN_TEMP_DAYS',
       'MIN_ANN_TEMP_AVG', 'MIN_ANN_TEMP_DAYS', 'MAX_ANN_TEMP', 'MIN_ANN_TEMP',
       'DAYS_ABOVE_32_C_YR', 'DAYS_BELOW_0_C_YR', 'FREEZE_INDEX_DAYS_YR',
       'FREEZE_THAW_DAYS_YR', 'MAX_CONTRIBUTE_WS

In [52]:
columns = [
    'SHRP_ID',#
    'STATE_CODE',#
    'YEAR',#
    'CONSTRUCTION_NO',
    'MAX_MEAN_DEPTH_1_8',#
    'RES_MOD_AVG',#
    'ANL_KESAL_LTPP_LN_YR',#
    'DAYS_ABOVE_32_C_YR',#
    'FREEZE_INDEX_YR',#
    'MEAN_ANN_TEMP_AVG',#
    'TOTAL_ANN_PRECIP',#
    'MR_MATL_TYPE'
]

nores_columns = [
    'SHRP_ID',#
    'STATE_CODE',#
    'YEAR',#
    'CONSTRUCTION_NO',
    'MAX_MEAN_DEPTH_1_8',#
    'ANL_KESAL_LTPP_LN_YR',#
    'DAYS_ABOVE_32_C_YR',#
    'FREEZE_INDEX_YR',#
    'MEAN_ANN_TEMP_AVG',#
    'TOTAL_ANN_PRECIP'#
]

In [53]:
allbutmix_merged = allbutmix_merged[columns]
allbutmix_merged

,SHRP_ID,STATE_CODE,YEAR,CONSTRUCTION_NO,MAX_MEAN_DEPTH_1_8,RES_MOD_AVG,ANL_KESAL_LTPP_LN_YR,DAYS_ABOVE_32_C_YR,FREEZE_INDEX_YR,MEAN_ANN_TEMP_AVG,TOTAL_ANN_PRECIP,MR_MATL_TYPE
0,1024,37,1992,1,9,21,80,2.0,11.0,13.5,1471.300049,2
1,1024,37,1992,1,9,19,80,2.0,11.0,13.5,1471.300049,2
2,1024,37,1992,1,9,77,80,2.0,11.0,13.5,1471.300049,2
3,1024,37,1992,1,9,79,80,2.0,11.0,13.5,1471.300049,2
4,1024,37,1992,1,9,95,80,2.0,11.0,13.5,1471.300049,2
...,...,...,...,...,...,...,...,...,...,...,...,...
21751,3082,28,1992,1,5,115,107,17.0,12.0,15.5,1240.800049,2
21752,3082,28,1992,1,5,89,107,17.0,12.0,15.5,1240.800049,2
21753,3082,28,1992,1,5,134,107,17.0,12.0,15.5,1240.800049,2
21754,3082,28,1992,1,5,117,107,17.0,12.0,15.5,1240.800049,2


In [45]:
allbutmix_merged.to_csv('excels/allbutmix')

In [46]:
#allbutmix_merged.groupby(['SHRP_ID', 'STATE_CODE', 'YEAR']).mean()

In [47]:
#allbutmix_merged.groupby(nores_columns).mean()

### tst_l05b

In [40]:
tst_l05b = pd.read_excel('excels1/tst_l05b.xlsx')
tst_l05b

,SHRP_ID,STATE_CODE,STATE_CODE_EXP,CONSTRUCTION_NO,LAYER_NO,DESCRIPTION,DESCRIPTION_EXP,LAYER_TYPE,LAYER_TYPE_EXP,REPR_THICKNESS,MATL_CODE_EXP,MATL_CODE,PROJECT_LAYER_CODE,LAYER_COMMENT1,LAYER_COMMENT1_EXP,LAYER_COMMENT2,LAYER_COMMENT2_EXP,LAYER_COMMENT3,LAYER_COMMENT3_EXP,COMMENT_NOTE,INV_LAYER_NO,INV_LAYER_NO_2,RECORD_STATUS
0,6064,27,Minnesota,3,11,1,Overlay,AC,Asphalt concrete layer,1.7,"Hot Mixed, Hot Laid AC, Dense Graded",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E
1,0601,29,Missouri,2,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E
2,0601,29,Missouri,2,2,5,Base Layer,GB,Unbound (granular) base,4.2,Crushed Stone,303,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,E
3,0601,29,Missouri,2,3,3,Original Surface Layer,PC,Portland cement concrete layer,9.2,Portland Cement Concrete (JRCP),5,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,E
4,0601,29,Missouri,3,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38584,0507,83,Manitoba,7,6,2,Seal Coat,AC,Asphalt concrete layer,0.0,Chip Seal,71,I,S,Layer was completely removed by milling.,NaN,NaN,NaN,NaN,NaN,6.0,NaN,E
38585,0507,83,Manitoba,7,7,1,Overlay,AC,Asphalt concrete layer,6.9,"Hot Mixed, Hot Laid AC, Dense Graded",1,K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E
38586,0507,83,Manitoba,7,8,10,Surface Treatment,AC,Asphalt concrete layer,0.2,Chip Seal,71,L,P,Material code for layer derived from lab. test...,Q,Layer thickness derived by measuring same mate...,Z,Other (Enter description in COMMENT_NOTE field.),DERIVED FROM SECTION 830508,NaN,NaN,E
38587,0507,83,Manitoba,7,9,2,Seal Coat,AC,Asphalt concrete layer,0.3,Chip Seal,71,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E


In [41]:
tst_l05b = tst_l05b[tst_l05b['LAYER_NO'] == 1]
tst_l05b

,SHRP_ID,STATE_CODE,STATE_CODE_EXP,CONSTRUCTION_NO,LAYER_NO,DESCRIPTION,DESCRIPTION_EXP,LAYER_TYPE,LAYER_TYPE_EXP,REPR_THICKNESS,MATL_CODE_EXP,MATL_CODE,PROJECT_LAYER_CODE,LAYER_COMMENT1,LAYER_COMMENT1_EXP,LAYER_COMMENT2,LAYER_COMMENT2_EXP,LAYER_COMMENT3,LAYER_COMMENT3_EXP,COMMENT_NOTE,INV_LAYER_NO,INV_LAYER_NO_2,RECORD_STATUS
1,0601,29,Missouri,2,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E
4,0601,29,Missouri,3,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E
7,0601,29,Missouri,4,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E
10,0659,29,Missouri,3,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E
15,0659,29,Missouri,4,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38552,0504,83,Manitoba,7,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Silty Clay with Gravel,132,A,P,Material code for layer derived from lab. test...,Z,Other (Enter description in COMMENT_NOTE field.),NaN,NaN,DERIVED FROM SECTION 830505,1.0,NaN,E
38561,0505,83,Manitoba,7,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Silty Clay with Gravel,132,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E
38570,0506,83,Manitoba,6,1,7,Subgrade,SS,Subgrade (untreated),NaN,Coarse-Grained Soil: Silty Sand,214,B,P,Material code for layer derived from lab. test...,Z,Other (Enter description in COMMENT_NOTE field.),NaN,NaN,DERIVED FROM SECTION 830507,1.0,NaN,E
38579,0507,83,Manitoba,7,1,7,Subgrade,SS,Subgrade (untreated),NaN,Coarse-Grained Soil: Silty Sand,214,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E


In [42]:
tst_l05b['MATL_CODE_EXP'].unique()

array(['Fine-Grained Soils: Sandy Clay',
       'Fine-Grained Soils: Fat Inorganic Clay',
       'Fine-Grained Soils: Silt',
       'Fine-Grained Soils: Lean Inorganic Clay',
       'Fine-Grained Soils: Lean Clay with Sand',
       'Coarse-Grained Soil: Clayey Sand',
       'Fine-Grained Soils: Silty Clay',
       'Coarse-Grained Soils: Poorly Graded Sand',
       'Fine-Grained Soils: Sandy Silty Clay',
       'Coarse-Grained Soils: Poorly Graded Sand with Silt',
       'Coarse-Grained Soil: Silty Sand',
       'Fine-Grained Soils: Silty Clay with Gravel',
       'Fine-Grained Soils: Sandy Silt',
       'Fine-Grained Soils: Sandy Lean Clay',
       'Fine-Grained Soils: Sandy Silty Clay with Gravel',
       'Fine-Grained Soils: Clay', 'Fine-Grained Soils: Clayey Silt',
       'Coarse-Grained Soil: Silty Sand with Gravel',
       'Fine-Grained Soils: Gravelly Fat Clay with Sand',
       'Coarse-Grained Soils: Poorly Graded Sand with Silt and Gravel',
       'Coarse-Grained Soils: Well-Gr

In [43]:
material_df = tst_l05b['MATL_CODE_EXP'].str.split(pat=": ", expand=True)

In [44]:
tst_l05b = pd.concat([tst_l05b, material_df], axis=1)
tst_l05b

,SHRP_ID,STATE_CODE,STATE_CODE_EXP,CONSTRUCTION_NO,LAYER_NO,DESCRIPTION,DESCRIPTION_EXP,LAYER_TYPE,LAYER_TYPE_EXP,REPR_THICKNESS,MATL_CODE_EXP,MATL_CODE,PROJECT_LAYER_CODE,LAYER_COMMENT1,LAYER_COMMENT1_EXP,LAYER_COMMENT2,LAYER_COMMENT2_EXP,LAYER_COMMENT3,LAYER_COMMENT3_EXP,COMMENT_NOTE,INV_LAYER_NO,INV_LAYER_NO_2,RECORD_STATUS,0,1
1,0601,29,Missouri,2,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Sandy Clay
4,0601,29,Missouri,3,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Sandy Clay
7,0601,29,Missouri,4,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Sandy Clay
10,0659,29,Missouri,3,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Sandy Clay
15,0659,29,Missouri,4,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Sandy Clay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38552,0504,83,Manitoba,7,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Silty Clay with Gravel,132,A,P,Material code for layer derived from lab. test...,Z,Other (Enter description in COMMENT_NOTE field.),NaN,NaN,DERIVED FROM SECTION 830505,1.0,NaN,E,Fine-Grained Soils,Silty Clay with Gravel
38561,0505,83,Manitoba,7,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Silty Clay with Gravel,132,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Silty Clay with Gravel
38570,0506,83,Manitoba,6,1,7,Subgrade,SS,Subgrade (untreated),NaN,Coarse-Grained Soil: Silty Sand,214,B,P,Material code for layer derived from lab. test...,Z,Other (Enter description in COMMENT_NOTE field.),NaN,NaN,DERIVED FROM SECTION 830507,1.0,NaN,E,Coarse-Grained Soil,Silty Sand
38579,0507,83,Manitoba,7,1,7,Subgrade,SS,Subgrade (untreated),NaN,Coarse-Grained Soil: Silty Sand,214,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Coarse-Grained Soil,Silty Sand


In [45]:
tst_l05b.columns

Index([           'SHRP_ID',         'STATE_CODE',     'STATE_CODE_EXP',
          'CONSTRUCTION_NO',           'LAYER_NO',        'DESCRIPTION',
          'DESCRIPTION_EXP',         'LAYER_TYPE',     'LAYER_TYPE_EXP',
           'REPR_THICKNESS',      'MATL_CODE_EXP',          'MATL_CODE',
       'PROJECT_LAYER_CODE',     'LAYER_COMMENT1', 'LAYER_COMMENT1_EXP',
           'LAYER_COMMENT2', 'LAYER_COMMENT2_EXP',     'LAYER_COMMENT3',
       'LAYER_COMMENT3_EXP',       'COMMENT_NOTE',       'INV_LAYER_NO',
           'INV_LAYER_NO_2',      'RECORD_STATUS',                    0,
                          1],
      dtype='object')

In [46]:
tst_l05b['MR_MATL_TYPE'] = np.where(tst_l05b['MATL_CODE_EXP'].str.startswith('Fine-'), 2, 1)
tst_l05b

,SHRP_ID,STATE_CODE,STATE_CODE_EXP,CONSTRUCTION_NO,LAYER_NO,DESCRIPTION,DESCRIPTION_EXP,LAYER_TYPE,LAYER_TYPE_EXP,REPR_THICKNESS,MATL_CODE_EXP,MATL_CODE,PROJECT_LAYER_CODE,LAYER_COMMENT1,LAYER_COMMENT1_EXP,LAYER_COMMENT2,LAYER_COMMENT2_EXP,LAYER_COMMENT3,LAYER_COMMENT3_EXP,COMMENT_NOTE,INV_LAYER_NO,INV_LAYER_NO_2,RECORD_STATUS,0,1,MR_MATL_TYPE
1,0601,29,Missouri,2,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Sandy Clay,2
4,0601,29,Missouri,3,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Sandy Clay,2
7,0601,29,Missouri,4,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Sandy Clay,2
10,0659,29,Missouri,3,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Sandy Clay,2
15,0659,29,Missouri,4,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Sandy Clay,113,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Sandy Clay,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38552,0504,83,Manitoba,7,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Silty Clay with Gravel,132,A,P,Material code for layer derived from lab. test...,Z,Other (Enter description in COMMENT_NOTE field.),NaN,NaN,DERIVED FROM SECTION 830505,1.0,NaN,E,Fine-Grained Soils,Silty Clay with Gravel,2
38561,0505,83,Manitoba,7,1,7,Subgrade,SS,Subgrade (untreated),NaN,Fine-Grained Soils: Silty Clay with Gravel,132,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Fine-Grained Soils,Silty Clay with Gravel,2
38570,0506,83,Manitoba,6,1,7,Subgrade,SS,Subgrade (untreated),NaN,Coarse-Grained Soil: Silty Sand,214,B,P,Material code for layer derived from lab. test...,Z,Other (Enter description in COMMENT_NOTE field.),NaN,NaN,DERIVED FROM SECTION 830507,1.0,NaN,E,Coarse-Grained Soil,Silty Sand,1
38579,0507,83,Manitoba,7,1,7,Subgrade,SS,Subgrade (untreated),NaN,Coarse-Grained Soil: Silty Sand,214,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,E,Coarse-Grained Soil,Silty Sand,1


In [57]:
onemore_merged = allbutmix_merged.merge(tst_l05b, on=['SHRP_ID', 'STATE_CODE', 'CONSTRUCTION_NO', 'MR_MATL_TYPE'], how='left')
onemore_merged

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'STATE_CODE_EXP_x', 'RECORD_STATUS_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,VISIT_NO,RUN_NUMBER,STATE_CODE_EXP_y,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS_y,MAX_MEAN_DEPTH_1_8,STATE_CODE_EXP_p,VWS_ID_p,TOTAL_ANN_PRECIP,TOTAL_SNOWFALL_YR,INTENSE_PRECIP_DAYS_YR,WET_DAYS_YR,NUM_ANN_PRECIP_DAYS,NUM_SNOWFALL_DAYS_YR,SNOW_COVERED_DAYS_YR,MAX_CONTRIBUTE_WS_NO_p,RECORD_STATUS_p,EXPANDED_STATUS_p,Unnamed: 0,STATE_CODE_EXP_t,VWS_ID_t,MEAN_ANN_TEMP_AVG,FREEZE_INDEX_YR,FREEZE_THAW_YR,MEAN_ANN_TEMP_DAYS,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,MIN_ANN_TEMP_AVG,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t,STATE_CODE_EXP_x,LAYER_NO_x,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,CON_PRESSURE,NOM_MAX_AXIAL_STRESS,LAB_CODE,LAB_CODE_EXP,SAMPLE_AREA_NO,LAYER_TYPE_x,LAYER_TYPE_EXP_x,MR_MATL_TYPE,MR_MATL_TYPE_EXP,APPLIED_MAX_AXIAL_LOAD_AVG,APPLIED_MAX_AXIAL_LOAD_STD,APPLIED_CYCLIC_LOAD_AVG,APPLIED_CYCLIC_LOAD_STD,APPLIED_CONTACT_LOAD_AVG,APPLIED_CONTACT_LOAD_STD,APPLIED_MAX_AXIAL_STRESS_AVG,APPLIED_MAX_AXIAL_STRESS_STD,APPLIED_CYCLIC_STRESS_AVG,APPLIED_CYCLIC_STRESS_STD,APPLIED_CONTACT_STRESS_AVG,APPLIED_CONTACT_STRESS_STD,DEF_LVDT_1_AVG,DEF_LVDT_1_STD,DEF_LVDT_2_AVG,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_AVG,RES_MOD_STD,TEST_DATE,RECORD_STATUS_x,STATE_CODE_EXP_y,LAYER_NO_y,DESCRIPTION,DESCRIPTION_EXP,LAYER_TYPE_y,LAYER_TYPE_EXP_y,REPR_THICKNESS,MATL_CODE_EXP,MATL_CODE,PROJECT_LAYER_CODE,LAYER_COMMENT1,LAYER_COMMENT1_EXP,LAYER_COMMENT2,LAYER_COMMENT2_EXP,LAYER_COMMENT3,LAYER_COMMENT3_EXP,COMMENT_NOTE,INV_LAYER_NO,INV_LAYER_NO_2,RECORD_STATUS_y,0,1
0,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,41.400002,41.400002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,146.899994,0.4,NaN,NaN,NaN,NaN,36.900002,NaN,4.1,NaN,0.25212,NaN,0.26464,NaN,0.25839,0.00013,0.001374,NaN,21,0.0,1992-11-18 00:00:00,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,27.600000,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,230.199997,0.7,NaN,NaN,NaN,NaN,57.799999,NaN,6.9,NaN,0.45397,NaN,0.47648,NaN,0.46523,0.00036,0.003122,NaN,19,0.0,1992-11-18 00:00:00,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,1,Material sample obtained from area adjacent to...,1,BA2,BS02,41.400002,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,245.899994,0.3,NaN,NaN,NaN,NaN,62.000000,NaN,6.9,NaN,0.10559,NaN,0.12616,NaN,0.11587,0.00015,0.000802,NaN,77,0.0,1992-11-18 00:00:00,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,

In [103]:
onemore_merged['REPR_THICKNESS'].describe()

count    515.000000
mean     143.359223
std       58.563022
min       15.000000
25%      132.000000
50%      144.000000
75%      198.000000
max      198.000000
Name: REPR_THICKNESS, dtype: float64

In [58]:
final_cols = [
    'STATE_CODE',
    'SHRP_ID',
    'YEAR',
    'CONSTRUCTION_NO',
    'MAX_MEAN_DEPTH_1_8',
    'ANL_KESAL_LTPP_LN_YR',
    'TOTAL_ANN_PRECIP',
    'FREEZE_INDEX_YR',
    'MAX_ANN_TEMP_AVG',
    'MAX_ANN_TEMP_DAYS',
    'DAYS_ABOVE_32_C_YR',
    'MR_MATL_TYPE',
    'RES_MOD_AVG',
    'REPR_THICKNESS'
]

onemore_merged[final_cols]

,STATE_CODE,SHRP_ID,YEAR,CONSTRUCTION_NO,MAX_MEAN_DEPTH_1_8,ANL_KESAL_LTPP_LN_YR,TOTAL_ANN_PRECIP,FREEZE_INDEX_YR,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,DAYS_ABOVE_32_C_YR,MR_MATL_TYPE,RES_MOD_AVG,REPR_THICKNESS
0,37,1024,1992,1,9,80,1471.300049,11.0,19.799999,366.0,2.0,2,21,NaN
1,37,1024,1992,1,9,80,1471.300049,11.0,19.799999,366.0,2.0,2,19,NaN
2,37,1024,1992,1,9,80,1471.300049,11.0,19.799999,366.0,2.0,2,77,NaN
3,37,1024,1992,1,9,80,1471.300049,11.0,19.799999,366.0,2.0,2,79,NaN
4,37,1024,1992,1,9,80,1471.300049,11.0,19.799999,366.0,2.0,2,95,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21751,28,3082,1992,1,5,107,1240.800049,12.0,22.100000,366.0,17.0,2,115,NaN
21752,28,3082,1992,1,5,107,1240.800049,12.0,22.100000,366.0,17.0,2,89,NaN
21753,28,3082,1992,1,5,107,1240.800049,12.0,22.100000,366.0,17.0,2,134,NaN
21754,28,3082,1992,1,5,107,1240.800049,12.0,22.100000,366.0,17.0,2,117,NaN


In [110]:
onemore_merged[onemore_merged['MR_MATL_TYPE'] == 2]

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,VISIT_NO,RUN_NUMBER,STATE_CODE_EXP_y,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS_y,MAX_MEAN_DEPTH_1_8,STATE_CODE_EXP_p,VWS_ID_p,TOTAL_ANN_PRECIP,TOTAL_SNOWFALL_YR,INTENSE_PRECIP_DAYS_YR,WET_DAYS_YR,NUM_ANN_PRECIP_DAYS,NUM_SNOWFALL_DAYS_YR,SNOW_COVERED_DAYS_YR,MAX_CONTRIBUTE_WS_NO_p,RECORD_STATUS_p,EXPANDED_STATUS_p,Unnamed: 0,STATE_CODE_EXP_t,VWS_ID_t,MEAN_ANN_TEMP_AVG,FREEZE_INDEX_YR,FREEZE_THAW_YR,MEAN_ANN_TEMP_DAYS,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,MIN_ANN_TEMP_AVG,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t,STATE_CODE_EXP_x,LAYER_NO_x,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,CON_PRESSURE,NOM_MAX_AXIAL_STRESS,LAB_CODE,LAB_CODE_EXP,SAMPLE_AREA_NO,LAYER_TYPE_x,LAYER_TYPE_EXP_x,MR_MATL_TYPE,MR_MATL_TYPE_EXP,APPLIED_MAX_AXIAL_LOAD_AVG,APPLIED_MAX_AXIAL_LOAD_STD,APPLIED_CYCLIC_LOAD_AVG,APPLIED_CYCLIC_LOAD_STD,APPLIED_CONTACT_LOAD_AVG,APPLIED_CONTACT_LOAD_STD,APPLIED_MAX_AXIAL_STRESS_AVG,APPLIED_MAX_AXIAL_STRESS_STD,APPLIED_CYCLIC_STRESS_AVG,APPLIED_CYCLIC_STRESS_STD,APPLIED_CONTACT_STRESS_AVG,APPLIED_CONTACT_STRESS_STD,DEF_LVDT_1_AVG,DEF_LVDT_1_STD,DEF_LVDT_2_AVG,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_AVG,RES_MOD_STD,TEST_DATE,RECORD_STATUS_x,STATE_CODE_EXP_y,LAYER_NO_y,DESCRIPTION,DESCRIPTION_EXP,LAYER_TYPE_y,LAYER_TYPE_EXP_y,REPR_THICKNESS,MATL_CODE_EXP,MATL_CODE,PROJECT_LAYER_CODE,LAYER_COMMENT1,LAYER_COMMENT1_EXP,LAYER_COMMENT2,LAYER_COMMENT2_EXP,LAYER_COMMENT3,LAYER_COMMENT3_EXP,COMMENT_NOTE,INV_LAYER_NO,INV_LAYER_NO_2,RECORD_STATUS_y,0,1
0,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,41.400002,41.400002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,146.899994,0.4,NaN,NaN,NaN,NaN,36.900002,NaN,4.1,NaN,0.25212,NaN,0.26464,NaN,0.25839,0.00013,0.001374,NaN,21,0.0,1992-11-18 00:00:00,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,27.600000,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,230.199997,0.7,NaN,NaN,NaN,NaN,57.799999,NaN,6.9,NaN,0.45397,NaN,0.47648,NaN,0.46523,0.00036,0.003122,NaN,19,0.0,1992-11-18 00:00:00,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,1,Material sample obtained from area adjacent to...,1,BA2,BS02,41.400002,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,245.899994,0.3,NaN,NaN,NaN,NaN,62.000000,NaN,6.9,NaN,0.10559,NaN,0.12616,NaN,0.11587,0.00015,0.000802,NaN,77,0.0,1992-11-18 00:00:00,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,

In [106]:
onemore_merged[final_cols][onemore_merged['SHRP_ID'] == '3082']

,STATE_CODE,SHRP_ID,YEAR,CONSTRUCTION_NO,MAX_MEAN_DEPTH_1_8,ANL_KESAL_LTPP_LN_YR,TOTAL_ANN_PRECIP,FREEZE_INDEX_YR,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,DAYS_ABOVE_32_C_YR,MR_MATL_TYPE,RES_MOD_AVG,REPR_THICKNESS
21726,28,3082,1992,1,5,107,1240.800049,12.0,22.1,366.0,17.0,2,87,NaN
21727,28,3082,1992,1,5,107,1240.800049,12.0,22.1,366.0,17.0,2,76,NaN
21728,28,3082,1992,1,5,107,1240.800049,12.0,22.1,366.0,17.0,2,117,NaN
21729,28,3082,1992,1,5,107,1240.800049,12.0,22.1,366.0,17.0,2,103,NaN
21730,28,3082,1992,1,5,107,1240.800049,12.0,22.1,366.0,17.0,2,81,NaN
21731,28,3082,1992,1,5,107,1240.800049,12.0,22.1,366.0,17.0,2,108,NaN
21732,28,3082,1992,1,5,107,1240.800049,12.0,22.1,366.0,17.0,2,102,NaN
21733,28,3082,1992,1,5,107,1240.800049,12.0,22.1,366.0,17.0,2,92,NaN
21734,28,3082,1992,1,5,107,1240.800049,12.0,22.1,366.0,17.0,2,111,NaN
21735,28,3082,1992,1,5,107,1240.800049,12.0,22.1,366.0,17.0,2,67,NaN


In [112]:
onemore_merged[final_cols].to_excel('onemore_merged.xlsx')

## Random Forest model

In [59]:
temp = onemore_merged.drop(['RES_MOD_AVG', 'REPR_THICKNESS'], axis=1)
temp

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,VISIT_NO,RUN_NUMBER,STATE_CODE_EXP_y,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS_y,MAX_MEAN_DEPTH_1_8,STATE_CODE_EXP_p,VWS_ID_p,TOTAL_ANN_PRECIP,TOTAL_SNOWFALL_YR,INTENSE_PRECIP_DAYS_YR,WET_DAYS_YR,NUM_ANN_PRECIP_DAYS,NUM_SNOWFALL_DAYS_YR,SNOW_COVERED_DAYS_YR,MAX_CONTRIBUTE_WS_NO_p,RECORD_STATUS_p,EXPANDED_STATUS_p,Unnamed: 0,STATE_CODE_EXP_t,VWS_ID_t,MEAN_ANN_TEMP_AVG,FREEZE_INDEX_YR,FREEZE_THAW_YR,MEAN_ANN_TEMP_DAYS,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,MIN_ANN_TEMP_AVG,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t,STATE_CODE_EXP_x,LAYER_NO_x,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,CON_PRESSURE,NOM_MAX_AXIAL_STRESS,LAB_CODE,LAB_CODE_EXP,SAMPLE_AREA_NO,LAYER_TYPE_x,LAYER_TYPE_EXP_x,MR_MATL_TYPE,MR_MATL_TYPE_EXP,APPLIED_MAX_AXIAL_LOAD_AVG,APPLIED_MAX_AXIAL_LOAD_STD,APPLIED_CYCLIC_LOAD_AVG,APPLIED_CYCLIC_LOAD_STD,APPLIED_CONTACT_LOAD_AVG,APPLIED_CONTACT_LOAD_STD,APPLIED_MAX_AXIAL_STRESS_AVG,APPLIED_MAX_AXIAL_STRESS_STD,APPLIED_CYCLIC_STRESS_AVG,APPLIED_CYCLIC_STRESS_STD,APPLIED_CONTACT_STRESS_AVG,APPLIED_CONTACT_STRESS_STD,DEF_LVDT_1_AVG,DEF_LVDT_1_STD,DEF_LVDT_2_AVG,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_STD,TEST_DATE,RECORD_STATUS_x,STATE_CODE_EXP_y,LAYER_NO_y,DESCRIPTION,DESCRIPTION_EXP,LAYER_TYPE_y,LAYER_TYPE_EXP_y,MATL_CODE_EXP,MATL_CODE,PROJECT_LAYER_CODE,LAYER_COMMENT1,LAYER_COMMENT1_EXP,LAYER_COMMENT2,LAYER_COMMENT2_EXP,LAYER_COMMENT3,LAYER_COMMENT3_EXP,COMMENT_NOTE,INV_LAYER_NO,INV_LAYER_NO_2,RECORD_STATUS_y,0,1
0,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,41.400002,41.400002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,146.899994,0.4,NaN,NaN,NaN,NaN,36.900002,NaN,4.1,NaN,0.25212,NaN,0.26464,NaN,0.25839,0.00013,0.001374,NaN,0.0,1992-11-18 00:00:00,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,27.600000,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,230.199997,0.7,NaN,NaN,NaN,NaN,57.799999,NaN,6.9,NaN,0.45397,NaN,0.47648,NaN,0.46523,0.00036,0.003122,NaN,0.0,1992-11-18 00:00:00,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,1,Material sample obtained from area adjacent to...,1,BA2,BS02,41.400002,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,245.899994,0.3,NaN,NaN,NaN,NaN,62.000000,NaN,6.9,NaN,0.10559,NaN,0.12616,NaN,0.11587,0.00015,0.000802,NaN,0.0,1992-11-18 00:00:00,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0

In [61]:
temp.drop_duplicates(inplace=True)
temp

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,VISIT_NO,RUN_NUMBER,STATE_CODE_EXP_y,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS_y,MAX_MEAN_DEPTH_1_8,STATE_CODE_EXP_p,VWS_ID_p,TOTAL_ANN_PRECIP,TOTAL_SNOWFALL_YR,INTENSE_PRECIP_DAYS_YR,WET_DAYS_YR,NUM_ANN_PRECIP_DAYS,NUM_SNOWFALL_DAYS_YR,SNOW_COVERED_DAYS_YR,MAX_CONTRIBUTE_WS_NO_p,RECORD_STATUS_p,EXPANDED_STATUS_p,Unnamed: 0,STATE_CODE_EXP_t,VWS_ID_t,MEAN_ANN_TEMP_AVG,FREEZE_INDEX_YR,FREEZE_THAW_YR,MEAN_ANN_TEMP_DAYS,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,MIN_ANN_TEMP_AVG,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t,STATE_CODE_EXP_x,LAYER_NO_x,TEST_NO,TEST_NO_EXP,FIELD_SET,LOC_NO,SAMPLE_NO,CON_PRESSURE,NOM_MAX_AXIAL_STRESS,LAB_CODE,LAB_CODE_EXP,SAMPLE_AREA_NO,LAYER_TYPE_x,LAYER_TYPE_EXP_x,MR_MATL_TYPE,MR_MATL_TYPE_EXP,APPLIED_MAX_AXIAL_LOAD_AVG,APPLIED_MAX_AXIAL_LOAD_STD,APPLIED_CYCLIC_LOAD_AVG,APPLIED_CYCLIC_LOAD_STD,APPLIED_CONTACT_LOAD_AVG,APPLIED_CONTACT_LOAD_STD,APPLIED_MAX_AXIAL_STRESS_AVG,APPLIED_MAX_AXIAL_STRESS_STD,APPLIED_CYCLIC_STRESS_AVG,APPLIED_CYCLIC_STRESS_STD,APPLIED_CONTACT_STRESS_AVG,APPLIED_CONTACT_STRESS_STD,DEF_LVDT_1_AVG,DEF_LVDT_1_STD,DEF_LVDT_2_AVG,DEF_LVDT_2_STD,DEF_LVDT_1_2_AVG,DEF_LVDT_1_2_STD,RES_STRAIN_AVG,RES_STRAIN_STD,RES_MOD_STD,TEST_DATE,RECORD_STATUS_x,STATE_CODE_EXP_y,LAYER_NO_y,DESCRIPTION,DESCRIPTION_EXP,LAYER_TYPE_y,LAYER_TYPE_EXP_y,MATL_CODE_EXP,MATL_CODE,PROJECT_LAYER_CODE,LAYER_COMMENT1,LAYER_COMMENT1_EXP,LAYER_COMMENT2,LAYER_COMMENT2_EXP,LAYER_COMMENT3,LAYER_COMMENT3_EXP,COMMENT_NOTE,INV_LAYER_NO,INV_LAYER_NO_2,RECORD_STATUS_y,0,1
0,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,41.400002,41.400002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,146.899994,0.4,NaN,NaN,NaN,NaN,36.900002,NaN,4.1,NaN,0.25212,NaN,0.26464,NaN,0.25839,0.00013,0.001374,NaN,0.0,1992-11-18 00:00:00,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,2,Material sample obtained from area adjacent to...,1,TP1,BS55,27.600000,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,230.199997,0.7,NaN,NaN,NaN,NaN,57.799999,NaN,6.9,NaN,0.45397,NaN,0.47648,NaN,0.46523,0.00036,0.003122,NaN,0.0,1992-11-18 00:00:00,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0,159.0,366.0,366.0,0.0,5.0,E,0.0,83945.0,North Carolina,371024,13.5,11.0,86.0,366.0,19.799999,366.0,7.2,366.0,32.799999,-11.8,2.0,86.0,366.0,366.0,5.0,E,0.0,North Carolina,1,1,Material sample obtained from area adjacent to...,1,BA2,BS02,41.400002,68.900002,1311,"Law Engineering, Atlanta, GA",NaN,1,Subgrade,2,Fine grained material,NaN,NaN,245.899994,0.3,NaN,NaN,NaN,NaN,62.000000,NaN,6.9,NaN,0.10559,NaN,0.12616,NaN,0.11587,0.00015,0.000802,NaN,0.0,1992-11-18 00:00:00,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37,North Carolina,1024,1992,E,6600,735,3300,368,80,37102403,1,North Carolina,04/10/1992,1,E,9,North Carolina,371024,1471.300049,0.0,39.0

In [64]:
temper_columns = [
    'STATE_CODE',
    'SHRP_ID',
    'YEAR',
    'CONSTRUCTION_NO',
    'MAX_MEAN_DEPTH_1_8',
    'ANL_KESAL_LTPP_LN_YR',
    'TOTAL_ANN_PRECIP',
    'FREEZE_INDEX_YR',
    'MAX_ANN_TEMP_AVG',
    'MAX_ANN_TEMP_DAYS',
    'DAYS_ABOVE_32_C_YR',
    'AADT_ALL_VEHIC_2WAY',
    'AADT_TRUCK_COMBO_2WAY',
    'AADT_ALL_VEHIC',
    'AADT_TRUCK_COMBO'
]

temptrfsection_merged[temper_columns]

,STATE_CODE,SHRP_ID,YEAR,CONSTRUCTION_NO,MAX_MEAN_DEPTH_1_8,ANL_KESAL_LTPP_LN_YR,TOTAL_ANN_PRECIP,FREEZE_INDEX_YR,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,DAYS_ABOVE_32_C_YR,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO
0,42,7037,1989,1,4,790,1039.199951,602.0,13.800000,365.0,0.0,15257,5896,6102,2358
1,44,7401,1989,1,3,186,1398.800049,495.0,14.800000,365.0,3.0,10000,1400,3700,510
2,42,9027,1989,1,3,1012,1230.599976,351.0,15.700000,365.0,8.0,15913,5070,6365,2028
3,23,7023,1989,1,5,774,1135.900024,728.0,12.900000,365.0,4.0,33540,3720,10900,1860
4,51,2564,1989,1,4,589,1580.400024,52.0,20.100000,365.0,26.0,39585,5235,15834,2094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12047,28,3083,2003,1,2,25,1493.800049,39.0,22.100000,365.0,40.0,0,0,0,33
12048,28,3085,1999,2,3,23,1330.000000,27.0,24.000000,365.0,70.0,0,0,0,29
12049,28,3085,2001,2,4,25,1805.500000,41.0,22.700001,365.0,42.0,0,0,0,31
12050,28,3085,2003,3,3,26,1496.099976,42.0,21.900000,365.0,34.0,0,0,0,33


In [65]:
temptrfsection_merged.drop_duplicates(inplace=True)
temptrfsection_merged

,STATE_CODE,STATE_CODE_EXP_x,SHRP_ID,YEAR,RECORD_STATUS_x,AADT_ALL_VEHIC_2WAY,AADT_TRUCK_COMBO_2WAY,AADT_ALL_VEHIC,AADT_TRUCK_COMBO,ANL_KESAL_LTPP_LN_YR,VISIT_NO,RUN_NUMBER,STATE_CODE_EXP_y,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS_y,MAX_MEAN_DEPTH_1_8,STATE_CODE_EXP_p,VWS_ID_p,TOTAL_ANN_PRECIP,TOTAL_SNOWFALL_YR,INTENSE_PRECIP_DAYS_YR,WET_DAYS_YR,NUM_ANN_PRECIP_DAYS,NUM_SNOWFALL_DAYS_YR,SNOW_COVERED_DAYS_YR,MAX_CONTRIBUTE_WS_NO_p,RECORD_STATUS_p,EXPANDED_STATUS_p,Unnamed: 0,STATE_CODE_EXP_t,VWS_ID_t,MEAN_ANN_TEMP_AVG,FREEZE_INDEX_YR,FREEZE_THAW_YR,MEAN_ANN_TEMP_DAYS,MAX_ANN_TEMP_AVG,MAX_ANN_TEMP_DAYS,MIN_ANN_TEMP_AVG,MIN_ANN_TEMP_DAYS,MAX_ANN_TEMP,MIN_ANN_TEMP,DAYS_ABOVE_32_C_YR,DAYS_BELOW_0_C_YR,FREEZE_INDEX_DAYS_YR,FREEZE_THAW_DAYS_YR,MAX_CONTRIBUTE_WS_NO_t,RECORD_STATUS_t,EXPANDED_STATUS_t
0,42,Pennsylvania,7037,1989,E,15257,5896,6102,2358,790,42703701,1,Pennsylvania,08/29/1989,1,E,4,Pennsylvania,427037,1039.199951,1033.0,25.0,179.0,365.0,365.0,0.0,6.0,E,0.0,102692.0,Pennsylvania,427037,7.5,602.0,109.0,365.0,13.800000,365.0,1.2,365.0,31.700001,-27.299999,0.0,165.0,365.0,365.0,6.0,E,0.0
1,44,Rhode Island,7401,1989,E,10000,1400,3700,510,186,44740101,1,Rhode Island,08/04/1989,1,E,3,Rhode Island,447401,1398.800049,725.0,32.0,184.0,365.0,365.0,0.0,6.0,E,0.0,102864.0,Rhode Island,447401,8.8,495.0,102.0,365.0,14.800000,365.0,2.7,365.0,33.599998,-22.000000,3.0,145.0,365.0,365.0,6.0,E,0.0
2,42,Pennsylvania,9027,1989,E,15913,5070,6365,2028,1012,42902701,1,Pennsylvania,07/24/1989,1,E,3,Pennsylvania,429027,1230.599976,398.0,26.0,178.0,365.0,365.0,0.0,7.0,E,0.0,102999.0,Pennsylvania,429027,10.1,351.0,96.0,365.0,15.700000,365.0,4.4,365.0,33.400002,-17.500000,8.0,134.0,365.0,365.0,7.0,E,0.0
3,23,Maine,7023,1989,E,33540,3720,10900,1860,774,23702301,1,Maine,08/10/1989,1,E,5,Maine,237023,1135.900024,854.0,25.0,171.0,365.0,365.0,0.0,5.0,E,0.0,51179.0,Maine,237023,7.3,728.0,93.0,365.0,12.900000,365.0,1.7,365.0,35.400002,-25.600000,4.0,152.0,365.0,365.0,5.0,E,0.0
4,51,Virginia,2564,1989,E,39585,5235,15834,2094,589,51256401,1,Virginia,10/12/1989,1,E,4,Virginia,512564,1580.400024,641.0,44.0,184.0,365.0,365.0,0.0,5.0,E,0.0,108655.0,Virginia,512564,15.1,52.0,56.0,365.0,20.100000,365.0,10.0,365.0,36.099998,-13.400000,26.0,62.0,365.0,365.0,5.0,E,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12047,28,Mississippi,3083,2003,E,0,0,0,33,25,28308312,1,Mississippi,02/25/2003,1,E,2,Mississippi,283083,1493.800049,5.0,41.0,154.0,365.0,365.0,0.0,7.0,E,0.0,58278.0,Mississippi,283083,16.0,39.0,70.0,365.0,22.100000,365.0,9.9,365.0,35.599998,-14.400000,40.0,72.0,365.0,365.0,7.0,E,0.0
12048,28,Mississippi,3085,1999,E,0,0,0,29,23,28308509,1,Mississippi,09/14/1999,2,E,3,Mississippi,283085,1330.000000,0.0,37.0,123.0,365.0,365.0,0.0,5.0,E,0.0,58581.0,Mississippi,283085,17.0,27.0,55.0,365.0,24.000000,365.0,10.0,365.0,38.099998,-14.300000,70.0,57.0,365.0,365.0,5.0,E,0.0
12049,28,Mississippi,3085,2001,E,0,0,0,31,25,28308510,1,Mississippi,03/05/2001,2,E,4,Mississippi,283085,1805.500000,15.0,44.0,145.0,365.0,365.0,0.0,5.0,E,0.0,58583.0,Mississippi,283085,16.0,41.0,68.0,365.0,22.700001,365.0,9.3,365.0,35.599998,-14.900000,42.0,70.0,365.0,365.0,5.0,E,0.0
12050,28,Mississippi,3085,2003,E,0,0,0,33,26,28308511,1,Mississippi,02/25/2003,3,E,3,Mississippi,283085,1496.099976,7.0,40.0,154.0,365.0,365.0,0.0,5.0,E,0.0,58585.0,Mississippi,283085,15.7,42.0,72.0,365.0,21.900000,365.0,9.4,365.0,35.099998,-14.800000,34.0,74.0,365.0,365.0,5.0,E,0.0


In [66]:
temptrfsection_merged.to_excel('temptrfsection_merged.xlsx')